In [ ]:
# Install packages 
import pandas as pd
import time
from transformers import pipeline
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import undetected_chromedriver as uc

In [ ]:
# URL of Forex Factory News
chrome_options = Options()
url = "https://www.forexfactory.com/news"
chrome_options.add_argument("--disable-blink-features=AutomationControlled") 

In [ ]:
ua = UserAgent()
chrome_options.add_argument(f"user-agent={ua.random}")

In [ ]:
driver = uc.Chrome()
driver.get(url)
time.sleep(3)

In [ ]:
# Wait for the button to be clickable
for _ in range(4): 
    more_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'flexMore')]")))
    driver.execute_script("arguments[0].scrollIntoView();", more_button)
    driver.execute_script("arguments[0].click();", more_button)
    time.sleep(3)



In [ ]:
response = driver.page_source
soup = BeautifulSoup(response, "html.parser")

In [ ]:
div = soup.find("div", id="ui-outer")
articles=[]
div2=div.find("div", id="ui-inner")
section=div2.find("section", class_="content news")
div3=section.find("div",class_="pagearrange__layout pagearrange__layout--arrangeable pagearrange__layout--zippable full")
div4=div3.find("div",class_="pagearrange__layout-row full")
ul=div4.find("ul",class_="body flexposts")
for var in ul.find_all("li"):
    var_attr=var.find('a')
    if var_attr and 'href' in var_attr.attrs:
        title = var_attr['href']
        driver.get("https://www.forexfactory.com/"+title)
        response2=driver.page_source
        soup2 = BeautifulSoup(response2, "html.parser")
        articles.append(soup2.find("li",class_="news__article").find("h1").text.strip())
        
    

In [ ]:
# Load FinBERT model
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")
# Function to classify sentiment using FinBERT
def classify_sentiment(text):
    result = finbert(text)[0] 
    return result["label"]  

In [ ]:
df = pd.read_csv("data/forex_news.csv")
df = pd.concat([df, pd.DataFrame({'news': articles, 'labels': list(map(classify_sentiment, articles))})], axis=0, ignore_index=True)
df = df.drop_duplicates().reset_index(drop=True)  

In [ ]:
df['news'] = df['news'].fillna('')
df=df[['news', 'labels']]
df.to_csv("data/forex_news.csv")

In [ ]:
driver.quit()